In [1]:
#-*-coding:utf-8-*-
import codecs
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import re

In [10]:
#将偏旁的输出文件以字典的方式呈现
infile = '/Users/wangwenhui/Desktop/论文/NER/FOFE_charembeddings_radicalembeddings/数据/偏旁output.txt'
fr = codecs.open(infile, 'r', encoding= 'utf-8')

component = {}

for line in fr.readlines():
    currList = line.strip().split(':') 
    
    if len(currList) < 2:
        continue
    # radical
    if not currList[2].isspace(): # fan_radical
        char = currList[0].decode('utf-8')
        component[char] = []
        component[char].append(currList[2])
    elif not currList[1].isspace(): # jian_radical
        char = currList[0].decode('utf-8')
        component[char] = []
        component[char].append(currList[1])
    else: # has not heen searched!
        continue
     
    #pianpang
    if char == currList[1]:
        component[char].append(char)
        component[char].append(char)
        continue
        
    if currList[-1].find('(') == -1: 
        component[char].append(currList[1])
        t = ' '.join(currList[-1]).split() #汉字部件构造
        component[char].append(t)

    else: #首尾分解查字
        component[char].append(currList[1])
        pos = currList[-1].find('(')
        t = ' '.join(currList[-1][:pos]).split()
#         if len(t) > 1:
#             if t[0] in [currList[1], currList[2]]:
#                 component[currList[0].decode('utf-8')].append(t)
                
#             elif t[1] in [currList[1], currList[2]]:
#                 component[currList[0].decode('utf-8')].append(t)
            
#         else:
        component[currList[0].decode('utf-8')].append(t)
           

fr.close()

In [11]:
#对于“首尾分解查字”这种情况，将非偏旁部分替换
for k, v in component.iteritems():
    print 'NO!', k, v
    if len(v[2]) == 2:
        print k, len(v[2])
        if v[2][0] != v[1] and v[2][0] in component:
            print '0'
            t = v[2][0]
            component[k][2].remove(t)
            component[k][2].insert(0, ' '.join(component[t][2]))
        if v[2][1] != v[1] and v[2][1] in component:
            print '1'
            p = component[k][2].pop()
            component[k][2].extend(component[p][2])
   

In [12]:
#radical是简体字
outfile = '/Users/wangwenhui/Desktop/论文/NER/FOFE_charembeddings_radicalembeddings/数据/简体偏旁processed_output.txt'
fw = codecs.open(outfile, 'w', encoding= 'utf-8')
for k, v in component.iteritems():
    print v
    string = k
    string += ':' + v[0]
    if len(v) > 2:
        string += ':' + ' '.join(v[2])
    else:
        string += ':' + v[0]
    fw.write(string + '\n')
fw.close()

In [14]:
#radical是繁体字
outfile = '/Users/wangwenhui/Desktop/论文/NER/FOFE_charembeddings_radicalembeddings/数据/繁体偏旁processed_output.txt'
fw = codecs.open(outfile, 'w', encoding= 'utf-8')
for k, v in component.iteritems():
    print k, v
    string = k
    string += ':' + v[0]
    if len(v) > 2 and len(v[2]) > 1:
        for i, x in enumerate(v[2]):
            if x == v[1]:
                v[2][i] = v[0] 
        string += ':' + ' '.join(v[2])
    else:
        string += ':' + v[0]
    fw.write(string + '\n')
fw.close()